Reading Standard calibration media extended

In [4]:
import pandas as pd

df = pd.read_excel('Standard calibration in culture media_extended.xlsx', sheet_name='Raw data', )
df.head()

,E0_Blank_culture media MH - moving average baseline,Unnamed: 1,Pyo 100 uM_MH - moving average baseline,Unnamed: 3,Unnamed: 4,Pyo 50 uM_MH - moving average baseline,Unnamed: 6,Unnamed: 7,Pyo 25 uM_MH - moving average baseline,Unnamed: 9,...,Unnamed: 32,Unnamed: 33,Pyo 0.25 uM_MH - moving average baseline,Unnamed: 35,Unnamed: 36,Pyo 0.1 uM_MH - moving average baseline,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41
0,V,µA,µA,µA,µA,µA,µA,µA,µA,µA,...,µA,µA,µA,µA,µA,µA,µA,µA,µA,µA
1,-0.600097,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,-0.595262,0.063593,0.098656,0.109374,0.115718,0.082966,0.110628,0.104234,0.088812,0.082304,...,0.079479,0.075286,0.075833,0.076125,0.079187,0.083927,0.078385,0.110979,0.051843,0.066755
3,-0.590427,0.054031,0.121318,0.130725,0.148006,0.099352,0.122679,0.113728,0.090737,0.073225,...,0.04127,0.070401,0.071166,0.054775,0.054862,0.065697,0.060739,0.079771,0.044581,0.064057
4,-0.585592,0.033967,0.136981,0.152074,0.173293,0.115738,0.113729,0.119721,0.085662,0.085147,...,0.038062,0.048015,0.045499,0.040424,0.044536,0.043967,0.064092,0.062562,0.030319,0.047358


### Dataset structure
- There are 42 signals, stored as column vectors
- The first signal, from the first column, represents the potential - denoted with $E$ and measured in $V$ (volts) - and this potential was applied in all experiments, ensuring the same potential window.
- The other signals represent the cureents - denoted with $I$ and measured in $\mu A$ (microampers) - characteristic to the initial potential $E$ and to the concentration values - denoted with $c$ and measured in $\mu M$ (micromolars), for which the experiment was performed.
- For each concentration were performed a number of readings, as shown bellow:

| $c$ [$\mu M$]| Number of readings |
| -------- | ------- |
| 0 | 1 (the blank signal)|
| 100 | 3 |
| 50 | 3 |
| 25 | 3 |
| 15 | 3 |
| 10 | 3 |
| 7.5 | 3 |
| 5 | 3 |
| 2.5 | 2 |
| 1 | 3 |
| 0.5 | 6 |
| 0.5 | 6 |
| 0.25 | 3 |
| 0.1 | 5 |

### The Problem we are trying to solve
- Create a ML model that predicts the concentration value (in $\mu M$) of a given voltammogram signal.
- The current signal, represented in the voltammogram, can be seen as a function of potential. Therefore, the entire problem can be written as:
$$c = I(E)$$

In [6]:
concentrations = [
    0,
    100,
    100,
    100,
    50,
    50,
    50,
    25,
    25,
    25,
    15,
    15,
    15,
    10,
    10,
    10,
    7.5,
    7.5,
    7.5,
    5,
    5,
    5,
    2.5,
    2.5,
    1,
    1,
    1,
    0.5,
    0.5,
    0.5,
    0.5,
    0.5,
    0.5,
    0.25,
    0.25,
    0.25,
    0.1,
    0.1,
    0.1,
    0.1,
    0.1
]
potential = df.iloc[1:, 0].values
currents = []

for col in df.columns[1:]:
    currents.append(df[col].values[1:])
    # Process potential and current as needed


plotting each voltammogram

In [ ]:
from matplotlib import pyplot as plt
import os

path = 'voltammograms'
if not os.path.exists(path):
    os.makedirs(path)

for i, current in enumerate(currents):
    peak = max(current[18:70])
    peak_voltage = potential[list(current).index(peak)]
# create a new figure for each plot and close it after saving
    plt.figure(figsize=(6, 4))
    plt.plot(potential, current)
    plt.plot(peak_voltage, peak, 'rx')
    plt.grid()
    plt.xlabel('Voltage (V)')
    plt.ylabel('Current (µA)')
    plt.title(f'Voltammogram {i},\nConcentration: {concentrations[i]} µM,\npeak at {round(peak, 2)} µA and {round(peak_voltage, 2)} V')
    fname = f'voltammograms/voltammogram_{i:02d}.jpg'
    plt.savefig(fname, bbox_inches='tight', dpi=200)
    plt.show()
    plt.close()


In [ ]:
from scipy.stats import linregress
import numpy as np

def get_features_category_b(potential, current):
    # index of max current (peak) 
    # we go relative to this point 
    peak_idx = np.argmax(current)

    # defining the three zones based on the peak: baseline, pre-preak and post-peak
    # we consider the first 15 points as baseline 
    baseline_end = 15
    base_slice = slice(0, baseline_end) 

    #extract data
    x_base = potential[base_slice]
    y_base = current[base_slice]

    # 1. Mean Baseline Current: avg of the flat part (before the reaction)
    mean_base = np.mean(y_base)

    # 2. Baseline slope: slope of the baseline (to see if there is a drift / tilt)
    # using linregress
    if len(x_base) > 1:
        slope, intercept, _, _, _ = linregress(x_base, y_base)
        
        # calc residuals (actual - predicted) to find noise
        y_pred = slope * x_base + intercept
        residuals = y_base - y_pred
        # 5. Baseline RMS noise: identify differences between actual point and the linear regression line
        rms_noise = np.sqrt(np.mean(residuals**2))
    else:
        slope = 0
        rms_noise = 0

    # 3. Pre-peak slope: calc how steep the climb is
    pre_start = max(0, peak_idx - 5)
    if peak_idx > pre_start:
        pre_slope, _, _, _, _ = linregress(potential[pre_start:peak_idx], 
                                           current[pre_start:peak_idx])
    else:
        pre_slope = 0

    # 4. post-peak slope: calc how steep the decline is
    post_end = min(len(current), peak_idx + 5)
    if post_end > peak_idx:
        post_slope, _, _, _, _ = linregress(potential[peak_idx:post_end], 
                                            current[peak_idx:post_end])
    else:
        post_slope = 0
    return {
        "Mean_Baseline": mean_base,
        "Baseline_Slope": slope,
        "Baseline_RMS": rms_noise,
        "Pre_Peak_Slope": pre_slope,
        "Post_Peak_Slope": post_slope
    }

In [13]:
# process for category B

features_list = []
for i, current in enumerate(currents):
    # fix data if necessary
    current = np.array(current, dtype=float)
    potential = np.array(potential, dtype=float)

    # get features
    features = get_features_category_b(potential, current)
    features['Signal_idx'] = i
    features_list.append(features)

df_task_b = pd.DataFrame(features_list)
print(df_task_b.head())
df_task_b.to_excel('features_category_b.xlsx', index=False, sheet_name='category_b_features')

   Mean_Baseline  Baseline_Slope  Baseline_RMS  Pre_Peak_Slope  \
0       0.014617       -0.653677      0.014604        2.904106   
1       0.172864        1.726892      0.053452      232.509016   
2       0.168606        1.106599      0.059024      284.827683   
3       0.152448        0.570968      0.051097      280.032193   
4       0.116645        0.832615      0.034097      340.811600   

   Post_Peak_Slope  Signal_idx  
0        -5.450238           0  
1      -341.298139           1  
2      -221.577604           2  
3      -266.980443           3  
4      -308.299437           4  
